# Importing libraries


In [1]:
!python3 -m pip install pattern
!python3 -m pip install krippendorff

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 58.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.5/89.5 kB 10.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 25.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.4/348.4 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pattern: filename=

In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

import pandas as pd
from nltk.cluster import KMeansClusterer
from pattern.nl import sentiment
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
import gensim
from gensim.corpora import Dictionary
from gensim.models import CoherenceModel
from sklearn.model_selection import train_test_split
from sklearn.semi_supervised import LabelPropagation
from sklearn.metrics import accuracy_score, classification_report

from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
import numpy as np
import krippendorff
from sklearn.metrics import cohen_kappa_score
from sklearn.linear_model import LogisticRegression

from scipy.sparse import vstack
from collections import Counter

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Importing dataset

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df_master = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/BSc AI Thesis/comments_new.csv')

<ipython-input-4-447e67fde9da>:1: DtypeWarning: Columns (10,11,12,13,14,15,16,17,18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df_master = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/BSc AI Thesis/comments_new.csv')


In [11]:
df = df_master.head(50000)

In [12]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/BSc AI Thesis/output.csv', index=False)

# Data cleaning

In [ ]:
fill_nan = ['Label 1', 'Label 2', 'Label 3', 'Label 4',
            'Label 5', 'Label 6', 'Label 7', 'Label 8',
            'Label 9', 'Label 10', 'Label 11', 'Label 12']

df[fill_nan] = df[fill_nan].fillna('None')

<ipython-input-69-fa9b17d7417d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[fill_nan] = df[fill_nan].fillna('None')


# Label distribution

In [ ]:
count_1 = Counter(df['Label 1'])
count_2 = Counter(df['Label 2'])

subjective_1 = count_1['subjective']
objective_1 = count_1['objective']

subjective_2 = count_2['subjective']
objective_2 = count_2['objective']

total = subjective_1 + objective_1

subjective_avg = (subjective_1 + subjective_2) / 2
objective_avg = (objective_1 + objective_2) / 2

subjective_percent = round((subjective_avg / total) * 100, 2)
objective_percent = round((objective_avg / total) * 100, 2)

In [ ]:
print(f"Subjective labels: {subjective_percent}%")
print(f"Objective labels: {objective_percent}%")

Subjective labels: 58.59%
Objective labels: 41.41%


# Krippendorff's alpha

In [ ]:
label_encoder = LabelEncoder()

# Converting labels to int for alpha calculation
annotator_1 = label_encoder.fit_transform(df["Label 1"].iloc[:100].astype(str))
annotator_2 = label_encoder.transform(df["Label 2"].iloc[:100].astype(str))
annotator_3 = label_encoder.transform(df["Label 3"].iloc[:100].astype(str))
annotator_4 = label_encoder.transform(df["Label 4"].iloc[:100].astype(str))
annotator_5 = label_encoder.transform(df["Label 5"].iloc[:100].astype(str))
annotator_6 = label_encoder.transform(df["Label 6"].iloc[:100].astype(str))
annotator_7 = label_encoder.transform(df["Label 7"].iloc[:100].astype(str))
annotator_8 = label_encoder.transform(df["Label 8"].iloc[:100].astype(str))
annotator_9 = label_encoder.transform(df["Label 9"].iloc[:100].astype(str))
annotator_10 = label_encoder.transform(df["Label 10"].iloc[:100].astype(str))
annotator_11 = label_encoder.transform(df["Label 11"].iloc[:100].astype(str))
annotator_12 = label_encoder.transform(df["Label 12"].iloc[:100].astype(str))

df_alpha = pd.DataFrame({"annotator_1": annotator_1, "annotator_2": annotator_2, "annotator_3": annotator_3, "annotator_4": annotator_4,
                         "annotator_5": annotator_5, "annotator_6": annotator_6, "annotator_7": annotator_7, "annotator_8": annotator_8,
                         "annotator_9": annotator_9, "annotator_10": annotator_10, "annotator_11": annotator_11, "annotator_12": annotator_12})

# Calculating alpha
alpha = krippendorff.alpha(reliability_data=df_alpha.values, level_of_measurement="nominal")

In [ ]:
print(f"Krippendorff's alpha: {round(alpha, 2)}")

Krippendorff's alpha: 0.41


# Data pre-processing

In [ ]:
tokeniser = WordPunctTokenizer()
stopwords = set(stopwords.words('dutch'))

preprocessed_text = []

for comment in df['text']:
  # Normalising the comments by converting to lowercase and removing numbers
  tokens = tokeniser.tokenize(comment.lower())
  tokens = [token for token in tokens if token.isalpha()]

  # Stopword removal
  tokens = [token for token in tokens if token not in stopwords]

  preprocessed_text.append(' '.join(tokens))

df['preprocessed text'] = preprocessed_text

<ipython-input-78-3723af488177>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['preprocessed text'] = preprocessed_text


# Objective vs. subjective: Clustering

## Case 1: Analysis on comment level

In [ ]:
# Feature extraction
vectoriser = TfidfVectorizer()
comment_vectors = vectoriser.fit_transform(preprocessed_text)

# Initial clustering based on comment
kmeans = KMeans(n_clusters=2, n_init='auto')
kmeans.fit(comment_vectors)

cluster_labels_case_1_init = kmeans.labels_

# Calculating average subjectivity for each cluster
cluster_sentiments = {0: [], 1: []}

for comment, cluster_label in zip(df['preprocessed text'], cluster_labels_case_1_init):
  sentiment_value = sentiment(comment)[1]
  cluster_sentiments[cluster_label].append(sentiment_value)

avg_subjectivity = {label: sum(values) / len(values) for label, values in cluster_sentiments.items()}

# Re-assingning clusters based on subjectivity
subjective_cluster, objective_cluster = (0, 1) if avg_subjectivity[0] > avg_subjectivity[1] else (1, 0)
cluster_labels_case_1 = [1 if label == subjective_cluster else 0 for label in cluster_labels_case_1_init]

## Case 2: Analysis on sentence level

In [ ]:
cluster_labels_case_2 = []

for comment in df['preprocessed text']:
  # Splitting the comment sentences into tokens
  sentences = nltk.sent_tokenize(comment)

  # Calculating subjectivity for sentences
  subjective_values = [sentiment(sentence)[1] for sentence in sentences]
  subjective_count = sum(1 for val in subjective_values if val > 0.5)
  objective_count = len(subjective_values) - subjective_count

  # Assigning clusters based on subjectivity
  cluster_labels_case_2.append(1 if subjective_count > objective_count else 0)

## Adding cluster labels to df

In [ ]:
df.loc[:, "Label Clustering: Case 1"] = pd.Series(cluster_labels_case_1, index=df.index).map({1: 'subjective', 0: 'objective'})
df.loc[:, "Label Clustering: Case 2"] = pd.Series(cluster_labels_case_2, index=df.index).map({1: 'subjective', 0: 'objective'})

<ipython-input-81-76f05928be02>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, "Label Clustering: Case 1"] = pd.Series(cluster_labels_case_1, index=df.index).map({1: 'subjective', 0: 'objective'})
<ipython-input-81-76f05928be02>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, "Label Clustering: Case 2"] = pd.Series(cluster_labels_case_2, index=df.index).map({1: 'subjective', 0: 'objective'})


## Comparing the two cases

In [ ]:
cohen_kappa = cohen_kappa_score(df['Label Clustering: Case 1'], df['Label Clustering: Case 2'])

In [ ]:
print(f"Cohen's kappa reliability between the 2 approaches: {round(cohen_kappa, 2)}")

Cohen's kappa reliability between the 2 approaches: 0.07


## Comparing case 1 to gold labels

In [ ]:
cohen_kappa_case_1 = cohen_kappa_score(df['Label Clustering: Case 1'].iloc[:100], df['Label 2'].iloc[:100])

In [ ]:
print(f"Cohen's kappa reliability between case 1 and the gold label: {round(cohen_kappa_case_1, 2)}")

Cohen's kappa reliability between case 1 and the gold label: -0.02


## Comparing case 2 to gold labels

In [ ]:
cohen_kappa_case_2 = cohen_kappa_score(df['Label Clustering: Case 2'].iloc[:100], df['Label 2'].iloc[:100])

In [ ]:
print(f"Cohen's kappa reliability between case 2 and the gold label: {round(cohen_kappa_case_2, 2)}")

Cohen's kappa reliability between case 2 and the gold label: 0.11


## Distribution of labels in case 1

In [ ]:
count_case_1 = Counter(df['Label Clustering: Case 1'])

subjective_count_1 = count_case_1['subjective']
objective_count_1 = count_case_1['objective']
total_1 = subjective_count_1 + objective_count_1

subjective_percent_1 = round((subjective_count_1 / total_1) * 100, 2)
objective_percent_1 = round((objective_count_1 / total_1) * 100, 2)

In [ ]:
print(f"Subjective labels case 1: {subjective_percent_1}%")
print(f"Objective labels case 1: {objective_percent_1}%")

Subjective labels case 1: 21.38%
Objective labels case 1: 78.62%


## Distribution of labels in case 2

In [ ]:
count_case_2 = Counter(df['Label Clustering: Case 2'])

subjective_count_2 = count_case_2['subjective']
objective_count_2 = count_case_2['objective']
total_2 = subjective_count_2 + objective_count_2

subjective_percent_2 = round((subjective_count_2 / total_2) * 100, 2)
objective_percent_2 = round((objective_count_2 / total_2) * 100, 2)

In [ ]:
print(f"Subjective labels case 2: {subjective_percent_2}%")
print(f"Objective labels case 2: {objective_percent_2}%")

Subjective labels case 2: 57.84%
Objective labels case 2: 42.16%


# Objective vs. subjective: Semi-supervised

In [ ]:
# Labeled-unlabeled split
labeled_data = df.iloc[:100]
unlabeled_data = df.iloc[100:]

# Feature extraction
X_labeled = vectoriser.fit_transform(labeled_data['preprocessed text'])
y_labeled = labeled_data['Label 2']
X_unlabeled = vectoriser.transform(unlabeled_data['preprocessed text'])

# Train-test split
X_labeled_train, X_labeled_test, y_labeled_train, y_labeled_test = train_test_split(X_labeled, y_labeled, test_size=0.2)

# Logistic regression base model
model = LogisticRegression()
model.fit(X_labeled_train, y_labeled_train)

# Prediciting test labels
test_predictions = model.predict(X_labeled_test)

In [ ]:
print(f"Accuracy: {accuracy_score(y_labeled_test, test_predictions)}" '\n')
print(f"Classification report:\n {classification_report(y_labeled_test, test_predictions)}")

Accuracy: 0.65

Classification report:
               precision    recall  f1-score   support

   objective       0.00      0.00      0.00         7
  subjective       0.65      1.00      0.79        13

    accuracy                           0.65        20
   macro avg       0.33      0.50      0.39        20
weighted avg       0.42      0.65      0.51        20



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Predicting labels
unlabeled_pred = model.predict(X_unlabeled)
unlabeled_probab = model.predict_proba(X_unlabeled)

# High confidence threshold
confidence = 0.9
confidence_index = np.where(np.max(unlabeled_probab, axis=1) > confidence)[0]
y_temp = unlabeled_pred[confidence_index]
X_temp = X_unlabeled[confidence_index]

# Retraining model on entire dataset
X_combined = vstack((X_labeled, X_temp))
y_combined = np.concatenate((y_labeled, y_temp))
model.fit(X_combined, y_combined)

# Predict labels for all the rows in the dataset
comment_vectors_semi = vectoriser.transform(df['preprocessed text'])
predictions = model.predict(comment_vectors_semi)

label_map = {'objective': 0, 'subjective': 1}
cluster_labels_semi = [label_map[label] for label in predictions]

# Add the new column to the DataFrame
df['Label Semi-Supervised'] = predictions

<ipython-input-128-c025c414c48d>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Label Semi-Supervised'] = predictions


## Comparing semi-supervised to gold labels

In [ ]:
cohen_kappa_semi = cohen_kappa_score(df['Label Semi-Supervised'].iloc[:100], df['Label 2'].iloc[:100])

In [ ]:
print(f"Cohen's kappa reliability between semi-supervised and gold labels: {round(cohen_kappa_semi, 2)}")

Cohen's kappa reliability between semi-supervised and gold labels: 0.98


## Comparing clustering case 1 to to semi-supervised

In [ ]:
cohen_kappa_case_1_semi = cohen_kappa_score(df['Label Clustering: Case 1'], df['Label Semi-Supervised'])

In [ ]:
print(f"Cohen's kappa reliability between clustering case 1 and semi-supervised labels: {round(cohen_kappa_case_1_semi, 2)}")

Cohen's kappa reliability between clustering case 1 and semi-supervised labels: -0.01


## Comparing clustering case 2 to to semi-supervised

In [ ]:
cohen_kappa_case_2_semi = cohen_kappa_score(df['Label Clustering: Case 2'], df['Label Semi-Supervised'])

In [ ]:
print(f"Cohen's kappa reliability between clustering case 2 and semi-supervised labels: {round(cohen_kappa_case_2_semi, 2)}")

Cohen's kappa reliability between clustering case 2 and semi-supervised labels: 0.01


## Distribution of labels in semi-supervised

In [ ]:
count_semi = Counter(df['Label Semi-Supervised'])

subjective_semi = count_semi['subjective']
objective_semi = count_semi['objective']
total_semi = subjective_semi + objective_semi

subjective_percent_semi = round((subjective_semi / total_semi) * 100, 2)
objective_percent_semi = round((objective_semi / total_semi) * 100, 2)

In [ ]:
print(f"Subjective labels semi-supervised: {subjective_percent_semi}%")
print(f"Objective labels semi-supervised: {objective_percent_semi}%")

Subjective labels semi-supervised: 96.31%
Objective labels semi-supervised: 3.69%


# Objective vs. subjective: Thread analysis

## Modeling thread structure

In [ ]:
threads_per_article = {}
cluster_labels = cluster_labels_case_2

for index, row in df.iterrows():
  comment_id = row['comment_id']
  parent_id = row['comment_parent_id']
  comment_type = cluster_labels[index]
  article_id = row['article_id']

  if article_id not in threads_per_article:
    threads_per_article[article_id] = {}

  if pd.isnull(parent_id) or parent_id not in threads_per_article[article_id]:
    threads_per_article[article_id][comment_id] = {'comment': preprocessed_text[index], 'replies': [], 'type': comment_type,
                                                   'subjective_count': 0, 'objective_count': 0}
  else:
    parent_thread = threads_per_article[article_id][parent_id]
    parent_thread['replies'].append({'comment': preprocessed_text[index], 'comment_id': comment_id, 'type': comment_type})

for article_id, threads in threads_per_article.items():
  for thread_id, thread in threads.items():
    thread['subjective_count'] = sum(reply['type'] == 1 for reply in thread['replies'])
    thread['objective_count'] = sum(reply['type'] == 0 for reply in thread['replies'])

## Function to assign the labels

In [ ]:
def predict_comment_type(threads_per_article, comment_id, article_id):
    if article_id not in threads_per_article or comment_id not in threads_per_article[article_id]:
        return 'None'

    thread = threads_per_article[article_id][comment_id]
    subjective_count = thread['subjective_count']
    objective_count = thread['objective_count']

    if subjective_count > objective_count:
        return 'subjective'
    elif subjective_count < objective_count:
        return 'objective'
    else:
        return 'subjective' if thread['type'] == 1 else 'objective'  # No clear majority, keep the original type


## Asigning labels based on replies

In [ ]:
df['Label Thread'] = df.apply(lambda row: predict_comment_type(threads_per_article, row['comment_id'], row['article_id']), axis=1)

<ipython-input-144-4060cd386388>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Label Thread'] = df.apply(lambda row: predict_comment_type(threads_per_article, row['comment_id'], row['article_id']), axis=1)


## Comparing thread analysis to clustering case 1 labels

In [ ]:
cohen_kappa_thread_1 = cohen_kappa_score(df['Label Clustering: Case 1'], df['Label Thread'])

In [ ]:
print(f"Cohen's kappa reliability between thread analysis and clustering case 1 labels: {round(cohen_kappa_thread_1, 2)}")

Cohen's kappa reliability between thread analysis and clustering case 1 labels: 0.02


## Comparing thread analysis to clustering case 2 labels

In [ ]:
cohen_kappa_thread_2 = cohen_kappa_score(df['Label Clustering: Case 2'], df['Label Thread'])

In [ ]:
print(f"Cohen's kappa reliability between thread analysis and clustering case 2 labels: {round(cohen_kappa_thread_2, 2)}")

Cohen's kappa reliability between thread analysis and clustering case 2 labels: 0.36


## Comparing thread analysis to semi-supervised labels

In [ ]:
cohen_kappa_thread_semi = cohen_kappa_score(df['Label Semi-Supervised'], df['Label Thread'])

In [ ]:
print(f"Cohen's kappa reliability between thread analysis and semi-supervised labels: {round(cohen_kappa_thread_semi, 2)}")

Cohen's kappa reliability between thread analysis and semi-supervised labels: 0.0


# Summary

In [ ]:
article_opinion_counts = {}

for article_id, threads in threads_per_article.items():
  article_opinion_counts[article_id] = {'subjective': 0, 'objective': 0}
  for thread_id, thread in threads.items():
    article_opinion_counts[article_id]['subjective'] += thread['subjective_count']
    article_opinion_counts[article_id]['objective'] += thread['objective_count']
    article_opinion_counts[article_id]['subjective'] += 1 if thread['type'] == 1 else 0
    article_opinion_counts[article_id]['objective'] += 1 if thread['type'] == 0 else 0

summary = "Discussion Summary:\n\n"

for article_id, threads in threads_per_article.items():
  opinion_counts = article_opinion_counts[article_id]
  total_opinions = opinion_counts['subjective'] + opinion_counts['objective']
  perc_subjective = opinion_counts['subjective'] / total_opinions * 100
  perc_objective = opinion_counts['objective'] / total_opinions * 100

  summary += f"Article ID: {article_id}\n"
  summary += f"Percentage of Subjective Opinions: {perc_subjective:.2f}%\n"
  summary += f"Percentage of Objective Opinions: {perc_objective:.2f}%\n\n"

  summary += "Thread Analysis:\n"
  for thread_id, thread in threads.items():
    summary += f"\tParent Comment: {thread['comment']} (Type: {'Subjective' if thread['type'] == 1 else 'Objective'})\n"
    summary += f"\t\tSubjective Replies: {thread['subjective_count']}\n"
    summary += f"\t\tObjective Replies: {thread['objective_count']}\n"

    for reply in thread['replies']:
      summary += f"\t\tReply Comment: {reply['comment']} (Type: {'Subjective' if reply['type'] == 1 else 'Objective'})\n"

  summary += "\n"

In [ ]:
with open("/content/drive/MyDrive/Colab Notebooks/BSc AI Thesis/summary.txt", "w") as file:
    file.write(summary)